In [ ]:
!pip install siibra siibra-jugex matplotlib nilearn

In [ ]:
import siibra
from nilearn import plotting
import matplotlib.pyplot as plt
%matplotlib notebook

### Define two points in MNI space

We start by specifying some points in the reference space. Here, we use the MNI space. We can find such point, amongst other possibilities, by clicking them in the [siibra-explorer](https://siibra-explorer.apps.hbp.eu/).

In [ ]:
pt_mni = siibra.Point(
    (-25.65, -2.75, -33.75), space="mni152")
view1 = plotting.plot_anat(
    cut_coords=tuple(pt_mni), 
    title="Point in MNI152 space")
view1.add_markers([tuple(pt_mni)])

pt_bb = pt_mni.warp('bigbrain')
atlas = siibra.atlases['human']
bigbrain = atlas.get_template("bigbrain")
view2 = plotting.plot_anat(
    bigbrain.fetch(), cut_coords=tuple(pt_bb), 
    title="Point in BigBrain space")
view2.add_markers([tuple(pt_bb)])

In [ ]:
point = siibra.Point((-25.65, -2.75, -33.75), space="mni152", sigma_mm=5)
pmaps = siibra.atlases['human'].get_map(
    space="mni152", parcellation="julich 2.9", maptype="continuous")
assignments = pmaps.assign(point)
a = assignments.iloc[0] # look at the first assignment
view = plotting.plot_stat_map( 
    pmaps.fetch(mapindex=a.MapIndex), cut_coords=tuple(point),
    cmap='viridis',
    title=f"{a.Region} ({a.Correlation:.2f})") 
view.add_markers([tuple(point)])

In [ ]:
# These are copy-pasted from the interactive atlas viewer:
points = siibra.PointSet(
    [
        "-25.650mm, -2.750mm, -33.750mm",
        "-37.350mm, -81.050mm, -6.300mm"
    ], 
    space='mni152', sigma_mm=5)

In [ ]:
plotting.view_markers(list(map(tuple,points)), ['red', 'cyan'], marker_size=10)  

### Assign brain regions to the 3D points

We assign the points to brain regions from the Julich-Brain cytoarchitectonic atlas, using a certain location tolerance of 5mm standard deviation.

In [ ]:
atlas = siibra.atlases['human']
pmaps = atlas.get_map(space="mni152",parcellation="julich",maptype="continuous")
assignments = pmaps.assign(points)
assignments[:10]

Let's look at the most probable area found for each point.

In [ ]:
# get the best assignment for each of the two points
a1 = assignments.query('Component==0').iloc[0]
a2 = assignments.query('Component==1').iloc[0]

statmap1 = pmaps.fetch(mapindex=a1.MapIndex)
statmap2 = pmaps.fetch(mapindex=a2.MapIndex)

view1 = plotting.plot_stat_map(
    statmap1, 
    title=f"{a1.Region} ({a1.Correlation:.2f})", 
    cmap='viridis', cut_coords=tuple(points[0]))
view2 = plotting.plot_stat_map(
    statmap2, 
    title=f"{a2.Region} ({a2.Correlation:.2f})", 
    cmap='viridis', cut_coords=tuple(points[1]))
view1.add_markers([tuple(points[0])])
view2.add_markers([tuple(points[1])])

### Look for white matter bundles connecting the regions

Next, we look at the most probable region associated to each point, considering them as a source and target region to investigate connectivity. In order to find white matter fibre bundles which are likely to connect them, we use the probability maps of the white matter fibre bundle parcellation, and assign both the source and target region from Julich-Brain to the fibre bundles. The intersection of the resulting bundles gives us those that are likely to provide connections from source to target.

In [ ]:
# get the probabilistic maps of long fibre bundles
bundlemaps = atlas.get_map("mni152","long bundles","continuous")

# assign the source and target region to the bundles, 
# using their probability maps found above.
bundles1, components1 = bundlemaps.assign(statmap1, msg=f"Find bundles touching {a1.Region}")
bundles2, components2 = bundlemaps.assign(statmap2, msg=f"Find bundles touching {a2.Region}")
                
# intersect the two sets of related fiber bundles
intersection = bundles1.merge(right=bundles2, how='inner', on=['Region'])
intersection

In [ ]:
# plot each of the bundles in the intersection (but no more than 3)
for i, row in intersection.iterrows():
    view = plotting.plot_glass_brain(
        bundlemaps.fetch(mapindex=row.MapIndex_x),
        title=f"{a1.Region} and {a2.Region} connected by {row.Region} ")
    view.add_overlay(statmap1)
    view.add_overlay(statmap2)
    view.add_markers([tuple(points[0]),tuple(points[1])])

### Find connectivity in terms of streamlines from DTI

We also investigate the connectivtiy of the two regions as measured by in-vivo imaging. To do so, we select the first region in the atlas, and search for connectivity profiles.

In [ ]:
# get an access token for EBRAINS - this requires you to register an EBRAINS account!
import os
os.environ['SIIBRA_ENABLE_DEVICE_FLOW'] = 'true'
siibra.fetch_ebrains_token()

In [ ]:
# get structural connectivity matrices for  Julich Brain
features = siibra.get_features(pmaps.parcellation, "StreamlineCounts")

# just use the first one here
conn = features[0]

# grep the connectivity profile of the source region that we used above
src_region = pmaps.decode_region(a1.Region)
profile = conn.matrix[src_region]

Create a bar plot of the 20 strongest connections in the connectivity profile.

In [ ]:
fig = plt.figure()
profile.sort_values(ascending=False)[:20].plot(kind='bar', grid=True)
fig.gca().set_title(f"Connection strengths from area {src_region.name}")